In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torchvision import transforms, datasets

In [16]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device('cuda' if USE_CUDA else 'cpu')  

In [17]:
EPOCHS = 40
BATCH_SIZE = 64

In [18]:
train_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST(
        './data',
        train=True,
        download=True,
        transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))
        ])),
    batch_size=BATCH_SIZE,
    shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST(
        './data',
        train=False,
        transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))
        ])),
    batch_size=BATCH_SIZE,
    shuffle=True
)

In [29]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)
        
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [34]:
model = CNN().to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [35]:
def train(model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output =  model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        
        if batch_idx % 200 == 0:
            print("Train Epoch: {} [{}/{} ({:.0f}%)]\t Loss: {:.6f}"
                 .format(epoch, batch_idx * len(data), len(train_loader.dataset),
                        100. * batch_idx / len(train_loader), loss.item()))

In [36]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
            
            # batch loss aggregate
            test_loss += F.cross_entropy(output, target, reduction='sum').item()
            
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
    
    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

In [37]:
for epoch in range(1, EPOCHS+1):
    train(model, train_loader, optimizer, epoch)
    test_loss, test_accuracy = evaluate(model, test_loader)
    
    print("[{}] Test Loss: {:.4f}, Accuracy: {:.2f}%".format(epoch, test_loss, test_accuracy))

Train Epoch: 1 [0/60000 (0%)]	 Loss: 2.404952
Train Epoch: 1 [12800/60000 (21%)]	 Loss: 1.307989
Train Epoch: 1 [25600/60000 (43%)]	 Loss: 0.994351
Train Epoch: 1 [38400/60000 (64%)]	 Loss: 0.859094
Train Epoch: 1 [51200/60000 (85%)]	 Loss: 0.718388
[1] Test Loss: 0.6558, Accuracy: 74.91%
Train Epoch: 2 [0/60000 (0%)]	 Loss: 0.849439
Train Epoch: 2 [12800/60000 (21%)]	 Loss: 0.671565
Train Epoch: 2 [25600/60000 (43%)]	 Loss: 0.979898
Train Epoch: 2 [38400/60000 (64%)]	 Loss: 0.801622
Train Epoch: 2 [51200/60000 (85%)]	 Loss: 0.719392
[2] Test Loss: 0.5728, Accuracy: 77.73%
Train Epoch: 3 [0/60000 (0%)]	 Loss: 0.993288
Train Epoch: 3 [12800/60000 (21%)]	 Loss: 0.636887
Train Epoch: 3 [25600/60000 (43%)]	 Loss: 0.751888
Train Epoch: 3 [38400/60000 (64%)]	 Loss: 0.706934
Train Epoch: 3 [51200/60000 (85%)]	 Loss: 0.692329
[3] Test Loss: 0.5197, Accuracy: 79.49%
Train Epoch: 4 [0/60000 (0%)]	 Loss: 0.701730
Train Epoch: 4 [12800/60000 (21%)]	 Loss: 0.445242
Train Epoch: 4 [25600/60000 (43%)

[28] Test Loss: 0.3291, Accuracy: 87.77%
Train Epoch: 29 [0/60000 (0%)]	 Loss: 0.501616
Train Epoch: 29 [12800/60000 (21%)]	 Loss: 0.542850
Train Epoch: 29 [25600/60000 (43%)]	 Loss: 0.315853
Train Epoch: 29 [38400/60000 (64%)]	 Loss: 0.504850
Train Epoch: 29 [51200/60000 (85%)]	 Loss: 0.362190
[29] Test Loss: 0.3325, Accuracy: 87.86%
Train Epoch: 30 [0/60000 (0%)]	 Loss: 0.491274
Train Epoch: 30 [12800/60000 (21%)]	 Loss: 0.277072
Train Epoch: 30 [25600/60000 (43%)]	 Loss: 0.265356
Train Epoch: 30 [38400/60000 (64%)]	 Loss: 0.338371
Train Epoch: 30 [51200/60000 (85%)]	 Loss: 0.297901
[30] Test Loss: 0.3284, Accuracy: 87.80%
Train Epoch: 31 [0/60000 (0%)]	 Loss: 0.375684
Train Epoch: 31 [12800/60000 (21%)]	 Loss: 0.480342
Train Epoch: 31 [25600/60000 (43%)]	 Loss: 0.589811
Train Epoch: 31 [38400/60000 (64%)]	 Loss: 0.596975
Train Epoch: 31 [51200/60000 (85%)]	 Loss: 0.412153
[31] Test Loss: 0.3240, Accuracy: 88.18%
Train Epoch: 32 [0/60000 (0%)]	 Loss: 0.437041
Train Epoch: 32 [12800/6